In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score
from statsmodels.tsa.arima.model import ARIMA

In [7]:
AMZN = pd.read_csv("/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks/aapl.us.txt")

In [8]:
AMZN['Date'] = pd.to_datetime(AMZN['Date'])
AMZN.set_index('Date', inplace=True)
AMZN.sort_index(inplace=True)

In [10]:
last_date_in_AMZN =AMZN.index.get_level_values('Date').max()
print(last_date_in_AMZN)

2017-11-10 00:00:00


In [11]:
most_recent_date_in_AMZN = AMZN.index.get_level_values('Date').max()
ten_years_prior_to_recent = most_recent_date_in_AMZN - pd.DateOffset(years=10)

AMZN = AMZN[AMZN.index.get_level_values('Date') >= ten_years_prior_to_recent]

In [12]:
last_date_in_AMZN = AMZN.index.get_level_values('Date').min()
print(last_date_in_AMZN)

2007-11-12 00:00:00


In [13]:
def df_transform(df):
    df["Daily Return"] = df["Close"].pct_change()

    df["7d_MA"] = df["Close"].transform(lambda x: x.rolling(window=7).mean())

    df["30d_MA"] = df["Close"].transform(lambda x: x.rolling(window=30).mean())

    df["Roll_vol"] = df["Daily Return"].transform(lambda x: x.rolling(window=30).std())

    return df

In [14]:
AMZN_new = df_transform(AMZN)

In [21]:
AMZN.head(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Open,High,Low,Close,Volume,OpenInt,Daily Return,7d_MA,30d_MA,Roll_vol
Date,,,,,,,,,,
1984-09-07,0.42388,0.42902,0.41874,0.42388,23220030,0,NaN,NaN,NaN,NaN
1984-09-10,0.42388,0.42516,0.41366,0.42134,18022532,0,-0.005992,NaN,NaN,NaN
1984-09-11,0.42516,0.43668,0.42516,0.42902,42498199,0,0.018228,NaN,NaN,NaN
1984-09-12,0.42902,0.43157,0.41618,0.41618,37125801,0,-0.029929,NaN,NaN,NaN
1984-09-13,0.43927,0.44052,0.43927,0.43927,57822062,0,0.055481,NaN,NaN,NaN
1984-09-14,0.44052,0.45589,0.44052,0.44566,68847968,0,0.014547,NaN,NaN,NaN
1984-09-17,0.45718,0.46357,0.45718,0.45718,53755262,0,0.025849,0.433219,NaN,NaN
1984-09-18,0.45718,0.46103,0.44052,0.44052,27136886,0,-0.036441,0.435596,NaN,NaN
1984-09-19,0.44052,0.44566,0.43157,0.43157,29641922,0,-0.020317,0.437057,NaN,NaN


In [21]:
close_prices = AMZN['Close']
n = len(close_prices)
split_point = int(n * 0.8)
def create_lag_features(series, lags=5):
    df = pd.DataFrame()
    for i in range(1, lags+1):
        df[f'lag_{i}'] = series.shift(i)
    df['target'] = series
    return df.dropna()

lr_data = create_lag_features(close_prices)
train_lr = lr_data.iloc[:split_point - 5]
test_lr = lr_data.iloc[split_point - 5:]

lr_model = LinearRegression()
lr_model.fit(train_lr.drop('target', axis=1), train_lr['target'])
pred_lr = lr_model.predict(test_lr.drop('target', axis=1))
mae_lr = mean_absolute_error(test_lr['target'], pred_lr)
print(mae_lr)

1.0973749686233458


In [22]:
def compute_indicators(df):
    df = df.copy()
    df['MA7'] = df['Close'].rolling(window=7).mean()
    df['MA30'] = df['Close'].rolling(window=30).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df = df.dropna()
    return df

features_df = compute_indicators(AMZN)
features_df = features_df.dropna()
X = features_df[['MA7', 'MA30', 'RSI']]
y = features_df['Close']

X_train = X.iloc[:split_point - (len(AMZN) - len(features_df))]
X_test = X.iloc[split_point - (len(AMZN) - len(features_df)):]
y_train = y.iloc[:split_point - (len(AMZN) - len(features_df))]
y_test = y.iloc[split_point - (len(AMZN) - len(features_df)):]

rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
pred_rf = rf_model.predict(X_test)
mae_rf = mean_absolute_error(y_test, pred_rf)
print(mae_rf)

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


10.975940218253987


In [24]:
# Use ARIMA(5,1,0) as a start — you can tune (p,d,q) later
close_prices = AMZN['Close']
split_point = int(len(close_prices) * 0.8)
train = close_prices.iloc[:split_point]
test = close_prices.iloc[split_point:]
arima_model = ARIMA(train, order=(5,1,0))
arima_result = arima_model.fit()
pred_arima = arima_result.forecast(steps=len(test))
mae_arima = mean_absolute_error(test, pred_arima)
print(mae_arima)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


20.31170129725727


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [26]:
!pip install backtesting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 9.2 MB/s eta 0:00:00


In [27]:
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np

class MLBasedStrategy(Strategy):
    def init(self):
        # Save the trained model
        self.model = self.__class__.model  # Access from static variable

    def next(self):
        # Get last 30 bars (to compute indicators)
        if len(self.data.Close) < 30:
            return  # not enough data yet

        close = pd.Series(self.data.Close[-31:])  # need one extra for diff

        # Compute features
        ma7 = close.rolling(window=7).mean().iloc[-1]
        ma30 = close.rolling(window=30).mean().iloc[-1]
        
        delta = close.diff()
        gain = delta.where(delta > 0, 0).rolling(14).mean().iloc[-1]
        loss = -delta.where(delta < 0, 0).rolling(14).mean().iloc[-1]
        rs = gain / loss if loss != 0 else 0
        rsi = 100 - (100 / (1 + rs)) if loss != 0 else 100

        # Prepare feature vector
        X_input = pd.DataFrame([[ma7, ma30, rsi]], columns=['MA7', 'MA30', 'RSI'])

        # Predict the next day's closing price
        predicted_price = self.model.predict(X_input)[0]
        current_price = self.data.Close[-1]

        # Simple signal logic
        if predicted_price > current_price and not self.position:
            self.buy()
        elif predicted_price < current_price and self.position:
            self.sell()

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...